<a href="https://colab.research.google.com/github/dariadementeva/NQ_Working/blob/main/mapillary_streetscapes_lux_city_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs the package
!pip install mapillary

INFO: pip is looking at multiple versions of turfpy to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 37.4 MB/s eta 0:00:00
  Created wheel for turfpy: filename=turfpy-0.0.7-py3-none-any.whl size=39094 sha256=d08c01c01a3bf1415a18ba7a7734028036ff7f51d9e8f33a543ca0d36036a3a3
  Stored in directory: /root/.cache/pip/wheels/7b/4a/e1/faedb15d6c6c5ca5944a1ff705cdd6b20350781259bc2c437c
Successfully built turfpy
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [ ]:
from collections import Counter
import json
import mapillary.interface as mly
import math
import pandas as pd
import pprint, re
import random
import requests
import requests
import time

In [ ]:
ACCESS_TOKEN = "MLY|25517726937837208|3f8625861dba933ed6b04631405be5bd"

# Luxembourg City (rough) bbox
WEST, SOUTH, EAST, NORTH = 6.05, 49.55, 6.20, 49.67

FIELDS = "id,geometry,captured_at,thumb_1024_url"
LIMIT = 2000  # keep at max = 2000

# define grids to get the images per grid

def grid_bboxes(w, s, e, n, nx=4, ny=4):
    dx = (e - w) / nx
    dy = (n - s) / ny
    for ix in range(nx):
        for iy in range(ny):
            yield (w + ix*dx, s + iy*dy, w + (ix+1)*dx, s + (iy+1)*dy)

def get_images_bbox(bbox, retries=5):
    url = "https://graph.mapillary.com/images"
    params = {
        "access_token": ACCESS_TOKEN,
        "bbox": ",".join(map(str, bbox)),
        "fields": FIELDS,
        "limit": LIMIT,
    }

    for attempt in range(retries + 1):
        try:
            r = requests.get(url, params=params, timeout=60)
            if r.status_code in (429, 500, 502, 503, 504):
                raise requests.HTTPError(f"{r.status_code} {r.text[:200]}", response=r)
            r.raise_for_status()
            return r.json()
        except Exception as ex:
            if attempt == retries:
                raise
            time.sleep((2 ** attempt) + random.random())

def paginate(first_json):
    out = []
    data = first_json
    while True:
        out.extend(data.get("data", []))
        nxt = data.get("paging", {}).get("next")
        if not nxt:
            break
        r = requests.get(nxt, timeout=60)
        if r.status_code in (429, 500, 502, 503, 504):
            time.sleep(2 + random.random())
            r = requests.get(nxt, timeout=60)
        r.raise_for_status()
        data = r.json()
    return out

all_images = []
for bbox in grid_bboxes(WEST, SOUTH, EAST, NORTH, nx=5, ny=5):
    js = get_images_bbox(bbox)
    imgs = paginate(js)
    all_images.extend(imgs)
    print("tile bbox", bbox, "->", len(imgs), "images")

# deduplicate by id (grid overlaps, grid border duplicates)
uniq = {img["id"]: img for img in all_images}
images = list(uniq.values())
print("Unique images, total:", len(images))

tile bbox (6.05, 49.55, 6.08, 49.574) -> 1892 images
tile bbox (6.05, 49.574, 6.08, 49.598) -> 1133 images
tile bbox (6.05, 49.598, 6.08, 49.622) -> 1536 images
tile bbox (6.05, 49.622, 6.08, 49.646) -> 1613 images
tile bbox (6.05, 49.646, 6.08, 49.67) -> 1874 images
tile bbox (6.08, 49.55, 6.11, 49.574) -> 1815 images
tile bbox (6.08, 49.574, 6.11, 49.598) -> 1944 images
tile bbox (6.08, 49.598, 6.11, 49.622) -> 1526 images
tile bbox (6.08, 49.622, 6.11, 49.646) -> 1604 images
tile bbox (6.08, 49.646, 6.11, 49.67) -> 1946 images
tile bbox (6.11, 49.55, 6.14, 49.574) -> 1898 images
tile bbox (6.11, 49.574, 6.14, 49.598) -> 1775 images
tile bbox (6.11, 49.598, 6.14, 49.622) -> 1865 images
tile bbox (6.11, 49.622, 6.14, 49.646) -> 1934 images
tile bbox (6.11, 49.646, 6.14, 49.67) -> 1938 images
tile bbox (6.14, 49.55, 6.17, 49.574) -> 1724 images
tile bbox (6.14, 49.574, 6.17, 49.598) -> 1795 images
tile bbox (6.14, 49.598, 6.17, 49.622) -> 1763 images
tile bbox (6.14, 49.622, 6.17, 49.6

In [ ]:
# save to the json file

with open("mapillary_luxembourg_bbox.json", "w", encoding="utf-8") as f:
    json.dump(images, f, ensure_ascii=False, indent=2)

print("Saved", len(images), "records to mapillary_luxembourg_bbox.json")

Saved 43081 records to mapillary_luxembourg_bbox.json


In [ ]:
JSON_PATH = "/content/mapillary_luxembourg_bbox.json"

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

print("Top-level type:", type(data))

def pick_items(data):
    if isinstance(data, list):
        return data
    if isinstance(data, dict):
        for k in ["features", "images", "items", "data", "results"]:
            v = data.get(k)
            if isinstance(v, list):
                print(f"Using data['{k}'] as records")
                return v
        if all(isinstance(v, dict) for v in data.values()):
            print("Using dict values as records (id -> obj)")
            return list(data.values())
    return []

items = pick_items(data)
print("Number of records:", len(items))
print("First record type:", type(items[0]) if items else None)
print("Top keys in first record:", list(items[0].keys())[:30] if items and isinstance(items[0], dict) else None)


Top-level type: <class 'list'>
Number of records: 43081
First record type: <class 'dict'>
Top keys in first record: ['id', 'geometry', 'captured_at', 'thumb_1024_url']


In [ ]:
pp = pprint.PrettyPrinter(depth=5, width=120)

if isinstance(data, dict):
    print("Top-level keys:", list(data.keys())[:50])
    print("GeoJSON type:", data.get("type"))
    if "features" in data:
        print("features type:", type(data["features"]), "len:", len(data["features"]))
        items = data["features"]

print("example")
pp.pprint(items[0])


example
{'captured_at': 1680175272403,
 'geometry': {'coordinates': [6.0612534, 49.5592706], 'type': 'Point'},
 'id': '135115332855948',
 'thumb_1024_url': 'https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/An_aOcle52VSckWX-8Aj-80wq10VoFE98EFRqZVAWSs8ejWfz9v8dPCDXZ4yfwNHGXUN_Exh43v5Ol-c-DnM68MUduU4QwYhcEU-yKf4KgGeB0JX-fLzSj4B4o11vFk1KYT6B9H_nyT4qqhWfVVcpw?stp=s1024x576&edm=AOnQwmMEAAAA&_nc_gid=OCaOMKjbGr4CxLCNFvhJtA&_nc_oc=AdnUTMO1eWSaSSXKb1LOEQ7qsrmFpL5OUwDqf5L21v86b_qE9R7eILq7mxuM2egkXWE&ccb=10-5&oh=00_AfqsUtro9aKF3d2O38w8TS_7oo1FtvIhTqduyCwdHAffow&oe=69A2C415&_nc_sid=201bca'}


In [ ]:
# Create a csv for geocoding, keep ids, lon and lats, timestamps and urls

JSON_PATH = "/content/mapillary_luxembourg_bbox.json"
OUT_CSV = "/content/mapillary_all_43k_coords.csv"

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

features = data["features"] if isinstance(data, dict) and "features" in data else data
print("Total features:", len(features))

rows = []
skipped = 0
missing_url = 0

for feat in features:
    image_id = ( # get ID
        feat.get("id") or
        feat.get("image_id") or
        feat.get("properties", {}).get("id")
    )

    geom = feat.get("geometry", {}) # get geometry
    coords = geom.get("coordinates")

    #
    thumb_url = ( # get url
        feat.get("thumb_1024_url") or
        feat.get("thumb_2048_url") or
        feat.get("url")
    )


    captured_at = ( # get timestamp
        feat.get("captured_at") or
        feat.get("properties", {}).get("captured_at") or
        feat.get("properties", {}).get("timestamp") or
        ""
    )

    if image_id is None or not isinstance(coords, list) or len(coords) != 2:
        skipped += 1
        continue

    if not thumb_url:
        missing_url += 1
        thumb_url = ""

    lon, lat = coords

    rows.append({
        "id": str(image_id),
        "captured_at": captured_at,
        "lon": float(lon),
        "lat": float(lat),
        "thumb_1024_url": thumb_url
    })

df = (
    pd.DataFrame(rows)
    .drop_duplicates(subset=["id"])
    .reset_index(drop=True)
)

df.to_csv(OUT_CSV, index=False)

print("Saved:", OUT_CSV)
print({
    "rows_written": len(df),
    "skipped_invalid": skipped,
    "missing_url": missing_url
})

df.head()


Total features: 43081
✅ Saved: /content/mapillary_all_43k_coords.csv
{'rows_written': 43081, 'skipped_invalid': 0, 'missing_url': 45}


,id,captured_at,lon,lat,thumb_1024_url
0,135115332855948,1680175272403,6.061253,49.559271,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...
1,209139725534508,1698313506346,6.061008,49.559129,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...
2,311225928352720,1698313553399,6.078481,49.562919,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...
3,385057477049231,1658824856830,6.050998,49.553090,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...
4,679104880390821,1692888443941,6.051723,49.553629,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...


In [ ]:
from google.colab import files
files.download("/content/mapillary_all_43k_coords.csv") # download in csv for geocoding

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Fetch the images


In [11]:
import pandas as pd

csv_path = "/content/sample_data/geo_all_lux.csv"
df = pd.read_csv(csv_path)

df.head()


/tmp/ipython-input-974583121.py:4: DtypeWarning: Columns (28,32,40,41,44,45,46,49,50,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,id,captured_at,lon,lat,thumb_1024_url,batch_id,address,place_id,licence,osm_type,...,commercial,historic,craft,man_made,neighbourhood,farm,residential,place,allotments,industrial
0,778445043542389,1681798905521,6.058698,49.591418,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...,1,"CR 163, Grevels-Barrière, Lorentzscheuer, Bert...",108642548,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,787405862846807,1681798845498,6.053924,49.597758,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...,1,"Route de Longwy, Grevels-Barrière, Lorentzsche...",108421508,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1351241788777971,1678778758505,6.060283,49.594162,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...,1,"CR 163, Grevels-Barrière, Lorentzscheuer, Bert...",108642548,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2233262170140424,1493733556070,6.050446,49.597099,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...,1,"N 5, Grevels-Barrière, Lorentzscheuer, Bertran...",108132672,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,157685697113031,1677654436569,6.051386,49.597229,https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/A...,1,"Route de Longwy, Grevels-Barrière, Lorentzsche...",108834957,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
for batch_id, batch_df in df.groupby("batch_id"):
    print(f"Batch {batch_id} → {len(batch_df)} rows")


Batch 1 → 5157 rows
Batch 2 → 5627 rows
Batch 3 → 8141 rows
Batch 4 → 8577 rows
Batch 5 → 8616 rows


In [13]:
MAPILLARY_TOKEN = "MLY|25517726937837208|3f8625861dba933ed6b04631405be5bd"

In [22]:
import requests
import time
import os
from tqdm import tqdm

# Create output folder
os.makedirs("images_batch_1", exist_ok=True)

# Filter batch 1
batch_1_df = df[df["batch_id"] == 1].dropna(subset=["thumb_1024_url"])

print(f"Downloading {len(batch_1_df)} images from batch 1")

for _, row in tqdm(
    batch_1_df.iterrows(),
    total=len(batch_1_df),
    desc="Downloading images"
):
    image_url = row["thumb_1024_url"]
    image_id = str(row["id"])

    file_path = f"images_batch_1/{image_id}.jpg"

    # Skip already downloaded images
    if os.path.exists(file_path):
        continue

    try:
        r = requests.get(image_url, timeout=10)
        if r.status_code == 200:
            with open(file_path, "wb") as f:
                f.write(r.content)
    except Exception as e:
        print(f"Error {image_id}: {e}")

    time.sleep(0.3)  # avoid rate limits


In [25]:
import os
import pandas as pd
import re

# Folder with existing images
IMAGE_DIR = "images_batch_1"

# Filter batch 1
batch_1_df = df[
    (df["batch_id"] == 1) &
    df["id"].notna() &
    df["image_name"].notna()
]

def safe_filename(name):
    name = str(name)
    return re.sub(r'[\\/*?:"<>|]', "_", name)

renamed = 0
skipped = 0
missing = 0

for _, row in batch_1_df.iterrows():
    old_name = f"{row['id']}.jpg"
    new_name = f"{safe_filename(row['image_name'])}.jpg"

    old_path = os.path.join(IMAGE_DIR, old_name)
    new_path = os.path.join(IMAGE_DIR, new_name)

    if not os.path.exists(old_path):
        missing += 1
        continue

    if os.path.exists(new_path):
        skipped += 1
        continue

    os.rename(old_path, new_path)
    renamed += 1

print(f"Renamed: {renamed}")
print(f"Skipped (already exists): {skipped}")
print(f"Missing original files: {missing}")


Renamed: 5157
Skipped (already exists): 0
Missing original files: 0


In [27]:
import shutil

shutil.make_archive(
    base_name="images_batch_1",  # output zip name (without .zip)
    format="zip",
    root_dir="images_batch_1"
)

from google.colab import files

files.download("images_batch_1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>